In [5]:
import pandas as pd
import numpy as np

df = pd.read_excel(r"P:\Indicadores\Diagnóstico\Premiações\Premiações Fora Bi\Premiação Ifood\2025\Outubro - 2025\Premiados Wiipo Ifood - Setembro 2025.xlsx")
cd = pd.read_excel(r"C:\Users\wallace.souza\Downloads\RH\CADASTRO.xlsx")
af = pd.read_excel(r"C:\Users\wallace.souza\Downloads\RH\AFASTAMENTO.xlsx")
pn = pd.read_excel(r"C:\Users\wallace.souza\Downloads\RH\PUNIÇÕES.xlsx")



In [6]:
cd = cd[cd["Descrição do Local"] =='DELIVERY']
cd["Data Afastamento"] = pd.to_datetime(cd["Data Afastamento"], errors="coerce")
cd = cd[(cd["Data Afastamento"] > "2025-09-30") | (cd["Data Afastamento"].isna())]
cd = cd[cd["Descrição (Situação)"] != "Demitido"]


In [7]:
df1 = df[['Cadastro', 'Loja', 'Nome','Função']]

In [8]:
cd = cd.merge(df1,on="Cadastro",how="left")

In [9]:
cd["Data Afastamento"] = pd.to_datetime(cd["Data Afastamento"], errors="coerce")

In [10]:
cd['Periodo'] = cd["Admissão"] + pd.Timedelta(days=90)


In [11]:
cd["Situação"] = np.where(cd["Periodo"] > "2025-08-31","Periodo de Experiência","Efetivado" )

In [12]:
cd = cd[['Cadastro', 'Apelido (Filial)', 'Nome_x',
       'Título Reduzido (Cargo)','Função', 'Data Afastamento',
       'Descrição (Situação)','Admissão',"Periodo","Situação"]]

In [13]:
af=af[af["Descrição (Situação)"].isin(['Atestado', 'Faltas'])]

In [14]:
cd = cd.merge(af[["Colaborador","Descrição (Situação)",'Início', 'Término']], left_on="Cadastro", right_on="Colaborador", how="left")

In [15]:
import pandas as pd

# garantir que as colunas são datetime
cd["Início"] = pd.to_datetime(cd["Início"], errors="coerce")
cd["Término"] = pd.to_datetime(cd["Término"], errors="coerce")

# calcular diferença em dias
cd["Afastamento dias"] = (cd["Término"] - cd["Início"]).dt.days + 1


In [16]:
cd["Afastamento Total"] = cd.groupby(["Colaborador","Descrição (Situação)_y"])["Afastamento dias"].transform("sum")

In [17]:
cd=cd.drop_duplicates(subset=["Cadastro","Descrição (Situação)_y"])

In [18]:
pn = pn[['Cadastro', 'Data','Descrição(Tip)', 'Descrição']]
pn = pn[pn["Descrição(Tip)"] == "ADVERTÊNCIA ESCRITA"]
pn = pn[['Cadastro', 'Data','Descrição']]

In [19]:
cd = cd.merge(pn, on="Cadastro", how="left")

In [20]:
cd =cd[['Cadastro', 'Apelido (Filial)', 'Nome_x', 'Título Reduzido (Cargo)',
       'Função', 'Data Afastamento', 'Descrição (Situação)_x', 'Admissão',
       'Periodo', 'Situação','Descrição (Situação)_y',
       'Início', 'Término', 'Afastamento dias', 'Afastamento Total', 'Data',
       'Descrição']]

In [21]:
cd.columns =[['Cadastro', 'Apelido (Filial)', 'Nome', 'Título Reduzido (Cargo)',
       'Função', 'Data Afastamento', 'Descrição (Situação)', 'Admissão',
       'Periodo', 'Situação','Faltas/Atestado',
       'Início', 'Término', 'Afastamento dias', 'Afastamento Total', 'Data advertência',
       'Descrição']]

In [29]:
af[af['Colaborador'] == 13970]

,Colaborador,Nome (Colaborador),Início,Término,Descrição (Situação),Situação
3570,13970,JALINE DE OLIVEIRA VIEIRA,2025-10-17,2025-10-17,Faltas,15
4796,13970,JALINE DE OLIVEIRA VIEIRA,2025-10-10,2025-10-10,Faltas,15
6010,13970,JALINE DE OLIVEIRA VIEIRA,2025-10-02,2025-10-02,Faltas,15


In [33]:
pn[pn['Cadastro'] == 13970]

,Cadastro,Data,Descrição
321,13970,2025-10-01,MÁ CONDUTA: SAIU ANTES DE TERMINAR SUA JORNADA...
322,13970,2025-10-01,FALTA INJUSTIFICADA EM 30/09/25.
323,13970,2025-10-03,SUSPENSÃO DE 1 DIA POR FALTAS INJUSTIFICADAS E...
324,13970,2025-10-21,ENVIOU ÁGUA COM GÁS SENDO SEM GÁS EM 18/10/25.
325,13970,2025-10-21,FALTA INJUSTIFICADA EM 17/10/2025.
326,13970,2025-10-23,ENVIOU MERCADORIA TROCADA EM 22/10/25.


In [24]:
cd.to_csv(r"C:\Users\wallace.souza\Downloads\ifood.csv",index=False, sep=";")